# Challenge 3

In this challenge we will work on the `Orders.csv` data set in the previous [Subsetting and Descriptive Stats lab](../../lab-subsetting-and-descriptive-stats/your-code/main.ipynb). In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

# Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [2]:
# import required libraries
import numpy as np
import pandas as pd

Next, import `Orders.csv` from the "subsetting" lab folder into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [3]:
# enter your code here
orders = pd.read_csv("../../lab-subsetting-and-descriptive-stats/your-code/Orders.csv")
orders

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397919,541904,581587,22613,2011,12,5,12,pack of 20 spaceboy napkins,12,2011-12-09 12:50:00,0.85,12680,France,10.20
397920,541905,581587,22899,2011,12,5,12,children's apron dolly girl,6,2011-12-09 12:50:00,2.10,12680,France,12.60
397921,541906,581587,23254,2011,12,5,12,childrens cutlery dolly girl,4,2011-12-09 12:50:00,4.15,12680,France,16.60
397922,541907,581587,23255,2011,12,5,12,childrens cutlery circus parade,4,2011-12-09 12:50:00,4.15,12680,France,16.60


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [6]:
# your code here
clients_class = orders.groupby('CustomerID').agg({'amount_spent':'sum'}).sort_values(by='amount_spent',ascending=False)
clients_class

,amount_spent
CustomerID,
14646,280206.02
18102,259657.30
17450,194550.79
16446,168472.50
14911,143825.06
...,...
17956,12.75
16454,6.90
14792,6.20


In [9]:
p75 = clients_class.quantile(.75)

p95 = clients_class.quantile(.95)
print(p75,p95)

amount_spent    1661.64
Name: 0.75, dtype: float64 amount_spent    5840.182
Name: 0.95, dtype: float64


In [22]:
float(p75)

1661.6400000000003

In [23]:
cutoffs = [0.0,float(p75),float(p95),clients_class['amount_spent'].max()]

In [24]:
cutoffs

[0.0, 1661.6400000000003, 5840.181999999983, 280206.01999999984]

In [25]:
labels=pd.cut(clients_class['amount_spent'],cutoffs,labels=['Regular client','Preferred','VIP'])
labels = pd.DataFrame(labels)
labels.columns=['Label']
labels

,Label
CustomerID,
14646,VIP
18102,VIP
17450,VIP
16446,VIP
14911,VIP
...,...
17956,Regular client
16454,Regular client
14792,Regular client


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

# Q2: How to identify which country has the most VIP Customers?

# Q3: How to identify which country has the most VIP+Preferred Customers combined?

Provide your solution for Q2 below:

In [26]:
# your code here

clients_label = labels.join(orders['Country'],on='CustomerID',how='left')

In [27]:
clients_label[clients_label['Label']=='VIP'].groupby('Country').count().sort_values(by='Label',ascending=False)

,Label
Country,
United Kingdom,191
France,6
Germany,6
Norway,6
EIRE,3
Portugal,2
Spain,2
Japan,1


In [28]:
clients_label[(clients_label['Label']=='VIP') | (clients_label['Label']=='Preferred')].groupby('Country').count().sort_values(by='Label',ascending=False)

,Label
Country,
United Kingdom,971
Norway,24
Germany,22
France,19
Spain,15
EIRE,12
Denmark,8
Japan,6
Portugal,6
